#Resources

* https://www.youtube.com/watch?v=ZzgUqFtxgXI
* https://www.youtube.com/watch?v=7aBRk_JP-qY
* https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/3/document-splitting

#Introduction

Title: Exam MCQ Generator

Game Plan:

*   Divide the document into multiple chuncks and then select chunks at random, pass them to Langchain and ask to make an MCQ out of it

#Installs

In [1]:
!pip install -q python-dotenv

!pip install -q langchain_experimental
!pip install -q langchain

!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requ

In [2]:
!pip install -q pydantic

#Imports

In [3]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

#Data Source

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
wizard_in_oz_text_path = '/content/drive/MyDrive/datasets/llm-from-scratch/wizard_in_oz.txt'

In [6]:
with open(wizard_in_oz_text_path, 'r', encoding='utf-8') as f:
  text = f.read()

In [7]:
print(text[:100])

﻿Title: Dorothy and the Wizard in Oz


Author: L. Frank Baum

Illustrator: John R. Neill

Release da


#Env

In [8]:
env_path = '/content/drive/MyDrive/credentials/data-analytics-demo/.env'

In [9]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(env_path)

True

In [11]:
GOOGLE_PALM_API_KEY = os.environ['GOOGLE_PALM_API_KEY']
HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']
OPEN_AI_API_KEY = os.environ['OPEN_AI_API_KEY']

# GOOGLE_PALM_API_KEY, HUGGINGFACE_API_KEY, OPEN_AI_API_KEY

#Model

In [93]:
from pydantic import BaseModel, Field

class MCQModel(BaseModel):
  question: str = Field(description="This is the question text")
  options: list[str] = Field(description="This is a list of multiple choices or options avalible")
  correct_option: str = Field(description="This is the correct choice or option")

In [94]:
from langchain.output_parsers import PydanticOutputParser

pydantic_parser = PydanticOutputParser(pydantic_object=MCQModel)

format_instructions = pydantic_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"question": {"title": "Question", "description": "This is the question text", "type": "string"}, "options": {"title": "Options", "description": "This is a list of multiple choices or options avalible", "type": "array", "items": {"type": "string"}}, "correct_option": {"title": "Correct Option", "description": "This is the correct choice or option", "type": "string"}}, "required": ["question", "options", "correct_option"]}
```


In [95]:
langchain_llm = ChatOpenAI(openai_api_key=OPEN_AI_API_KEY, model_name="gpt-3.5-turbo-0613", verbose=False,)

In [96]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [97]:
texts = text_splitter.create_documents([text])

In [98]:
print(f'Total Documents Created: {len(texts)}')

Total Documents Created: 287


In [99]:
prompt = (
    "Text:"
    "\n{text}"
    "\nGenerate an Multiple Choice Question from this text. Also return the correct option."
    "\nDo not use any outside information"
    "\n{format_instructions}"
)

In [100]:
prompt_template = ChatPromptTemplate.from_template(prompt)

prompt_template

ChatPromptTemplate(input_variables=['format_instructions', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], template='Text:\n{text}\nGenerate an Multiple Choice Question from this text. Also return the correct option.\nDo not use any outside information\n{format_instructions}'))])

In [101]:
llm_chain = LLMChain(llm=langchain_llm, prompt=prompt_template, output_key="result")

In [102]:
from langchain.callbacks import get_openai_callback as get_langchain_openai_callback
import random

def get_question():
  doc_index = random.randint(0, len(texts)-1)

  with get_langchain_openai_callback() as cb_langchain:
    response = llm_chain.invoke({"text": texts[doc_index], "format_instructions": format_instructions})

  return {'doc_index': doc_index, 'mcq': response['result'], 'cb_langchain': cb_langchain}

#Testing

In [109]:
response_raw = get_question()
response_parsed = MCQModel.parse_raw(response_raw['mcq'])

In [110]:
print(response_parsed)

print('\n')
print(f"doc_index = {response_raw['doc_index']}")
print('\ncosting:\n')
print(f"{response_raw['cb_langchain']}")

question='Who grumbled about being an ordinary horse in a fairy country?' options=['Jim', 'Zeb', 'Dorothy', 'Ozma'] correct_option='Jim'


doc_index = 258

costing:

Tokens Used: 564
	Prompt Tokens: 520
	Completion Tokens: 44
Successful Requests: 1
Total Cost (USD): $0.0008680000000000001


In [111]:
response_parsed

MCQModel(question='Who grumbled about being an ordinary horse in a fairy country?', options=['Jim', 'Zeb', 'Dorothy', 'Ozma'], correct_option='Jim')

#Pydantic Demo

In [90]:
from pydantic import BaseModel, EmailStr, validator

class User(BaseModel):
  name: str
  # email: EmailStr
  email: str
  account_id: int

  @validator("account_id")
  def validate_account_id(cls, value):
    if value<=0:
      raise ValueError(f'Account ID cannot be negative: {value}')

    return value

user = User(name='jack', email='a@b.com', account_id=1234)

print(user)
print(user.json())
print(user.dict())
print(user.parse_raw(user.json()))

name='jack' email='a@b.com' account_id=1234
{"name": "jack", "email": "a@b.com", "account_id": 1234}
{'name': 'jack', 'email': 'a@b.com', 'account_id': 1234}
name='jack' email='a@b.com' account_id=1234
